In [5]:
import ccxt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#binance api를 이용해 코인 시간봉 받기
binance = ccxt.binance()

btc_ohlcv = binance.fetch_ohlcv("BTC/USDT", '1h')
df = pd.DataFrame(btc_ohlcv, columns=['datetime', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
df.set_index('datetime', inplace=True)
df.head()

,Open,High,Low,Close,Volume
datetime,,,,,
2022-10-12 00:00:00,19060.00,19132.93,19024.65,19092.17,10190.57131
2022-10-12 01:00:00,19092.17,19101.23,19030.63,19063.24,6431.61275
2022-10-12 02:00:00,19063.24,19080.61,19030.91,19066.42,5740.60223
2022-10-12 03:00:00,19065.69,19090.15,19042.61,19048.73,4711.28460
2022-10-12 04:00:00,19049.09,19123.38,19041.90,19099.51,7695.11601


In [6]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
import talib

#전략 설정
class RSI(Strategy):

    upper_bound = 70
    lower_bound = 30
    rsi_window = 14


    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_window)
    
    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()

        elif crossover(self.lower_bound, self.rsi):
            self.buy()

In [7]:
bt = Backtest(df, RSI, cash=50000, commission=.0015)
stats = bt.run()
stats

Start                     2022-10-12 00:00:00
End                       2022-11-01 19:00:00
Duration                     20 days 19:00:00
Exposure Time [%]                         3.2
Equity Final [$]                  52286.43168
Equity Peak [$]                   52286.47168
Return [%]                           4.572863
Buy & Hold Return [%]                7.193892
Return (Ann.) [%]                  117.530777
Volatility (Ann.) [%]               29.424238
Sharpe Ratio                         3.994353
Sortino Ratio                             inf
Calmar Ratio                        79.980112
Max. Drawdown [%]                     -1.4695
Avg. Drawdown [%]                   -0.472357
Max. Drawdown Duration       18 days 18:00:00
Avg. Drawdown Duration        4 days 20:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       6.107746
Worst Trade [%]                      6.107746
Avg. Trade [%]                    

In [8]:
bt.plot()

Row(id='2680', ...)

In [9]:
stats = bt.optimize(upper_bound = range(55, 85, 5), 
                    lower_bound = range(10, 45, 5),
                    rsi_window = range(10, 30, 2),
                    maximize = 'Return [%]',
                    constraint = lambda param: param.upper_bound > param.lower_bound)
stats

  0%|          | 0/12 [00:00<?, ?it/s]

Start                     2022-10-12 00:00:00
End                       2022-11-01 19:00:00
Duration                     20 days 19:00:00
Exposure Time [%]                        26.4
Equity Final [$]                  54860.27358
Equity Peak [$]                   54860.27358
Return [%]                           9.720547
Buy & Hold Return [%]                7.193892
Return (Ann.) [%]                  401.467967
Volatility (Ann.) [%]                76.92381
Sharpe Ratio                         5.219034
Sortino Ratio                             inf
Calmar Ratio                       237.701445
Max. Drawdown [%]                   -1.688959
Avg. Drawdown [%]                   -0.394792
Max. Drawdown Duration        6 days 00:00:00
Avg. Drawdown Duration        1 days 05:00:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                       6.107746
Worst Trade [%]                      2.492486
Avg. Trade [%]                    

In [10]:
stats._strategy

<Strategy RSI(upper_bound=65,lower_bound=35,rsi_window=16)>

In [11]:
bt.plot()

Row(id='3885', ...)